AWS GET CLI Details

In [7]:
import os
import boto3
import pandas as pd
from dotenv import load_dotenv
load_dotenv()


os.environ['AWS_DEFAULT_REGION'] = os.getenv('AWS_DEFAULT_REGION')
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')

print("REGION:", os.environ['AWS_DEFAULT_REGION'])
print("KEY ID:", os.environ['AWS_ACCESS_KEY_ID'][:4] + '…')

# Use boto3 to interact with AWS
ec2 = boto3.client('ec2')
print("Default region:", ec2.meta.region_name)
print("Regions list:", [r['RegionName'] for r in ec2.describe_regions()['Regions']])

REGION: ap-south-1
KEY ID: AKIA…
Default region: ap-south-1
Regions list: ['ap-south-1', 'eu-north-1', 'eu-west-3', 'eu-west-2', 'eu-west-1', 'ap-northeast-3', 'ap-northeast-2', 'ap-northeast-1', 'ca-central-1', 'sa-east-1', 'ap-southeast-1', 'ap-southeast-2', 'eu-central-1', 'us-east-1', 'us-east-2', 'us-west-1', 'us-west-2']


In [ ]:
# # Cell 2: define a function to gather instance metadata per region
# def collect_ec2_details():
#     records = []
#     ec2_default = boto3.client('ec2')
#     regions = [r['RegionName'] for r in ec2_default.describe_regions()['Regions']]

#     for region in regions:
#         ec2 = boto3.client('ec2', region_name=region)
#         paginator = ec2.get_paginator('describe_instances')
#         for page in paginator.paginate():
#             for res in page.get('Reservations', []):
#                 for inst in res.get('Instances', []):
#                     records.append({
#                         'Region': region,
#                         'InstanceId': inst.get('InstanceId'),
#                         'InstanceType': inst.get('InstanceType'),
#                         'State': inst.get('State', {}).get('Name'),
#                         'LaunchTime': inst.get('LaunchTime')
#                     })

#     return pd.DataFrame(records)

In [ ]:
# # Cell 5: enrich with vCPU & memory specs
# def collect_ec2_with_specs():
#     records = []
#     ec2_default = boto3.client('ec2')
#     for region in [r['RegionName'] for r in ec2_default.describe_regions()['Regions']]:
#         client = boto3.client('ec2', region_name=region)
#         for page in client.get_paginator('describe_instances').paginate():
#             for res in page.get('Reservations', []):
#                 for inst in res.get('Instances', []):
#                     itype = inst['InstanceType']
#                     specs = client.describe_instance_types(InstanceTypes=[itype])['InstanceTypes'][0]
#                     records.append({
#                         'Region': region,
#                         'InstanceId': inst['InstanceId'],
#                         'InstanceType': itype,
#                         'State': inst['State']['Name'],
#                         'VCpu': specs['VCpuInfo']['DefaultVCpus'],
#                         'MemoryMiB': specs['MemoryInfo']['SizeInMiB']
#                     })
#     return pd.DataFrame(records)


In [ ]:
# # Cell 6: to fetch raw list
# df_instances = collect_ec2_details()
# df_instances.head()

# # Cell 7: to fetch enriched data
# df_specs = collect_ec2_with_specs()
# df_specs.head()


,Region,InstanceId,InstanceType,State,VCpu,MemoryMiB
0,ap-south-1,i-03f9d2f43e1058747,c7g.large,stopped,2,4096
1,ap-south-1,i-0273f9527356f2e4c,t2.micro,stopped,1,1024
2,ap-south-1,i-05f47b527f9c0c017,t3a.medium,running,2,4096
3,ap-south-1,i-0b5b0f29c98b01927,t4g.medium,stopped,2,4096
4,ap-south-1,i-057bcce818c28343d,t4g.medium,running,2,4096


In [ ]:
# # Cell 3: gather every instance type spec, region-by-region
# def collect_all_instance_types():
#     rows = []
#     ec2 = boto3.client('ec2')
#     regions = [r['RegionName'] for r in ec2.describe_regions()['Regions']]

#     for region in regions:
#         client = boto3.client('ec2', region_name=region)
#         paginator = client.get_paginator('describe_instance_types')
#         for page in paginator.paginate():
#             for it in page['InstanceTypes']:
#                 rows.append({
#                     'Region':                 region,
#                     'InstanceType':           it['InstanceType'],
#                     'vCPUs':                  it['VCpuInfo']['DefaultVCpus'],
#                     'MemoryMiB':              it['MemoryInfo']['SizeInMiB'],
#                     'StorageInfo':            it.get('InstanceStorageInfo'),
#                     'Accelerators':           it.get('GpuInfo') or it.get('FpgaInfo')
#                 })
#     return pd.DataFrame(rows)


In [ ]:
# Cell 3: gather every instance type spec, region-by-region (with RAM and NetworkPerformance)
def collect_all_instance_types():
    rows = []
    ec2_default = boto3.client('ec2')
    regions = [r['RegionName'] for r in ec2_default.describe_regions()['Regions']]

    for region in regions:
        client = boto3.client('ec2', region_name=region)
        paginator = client.get_paginator('describe_instance_types')
        for page in paginator.paginate():
            for it in page['InstanceTypes']:
                mem_mib = it['MemoryInfo']['SizeInMiB']
                ram_gib = round(mem_mib / 1024, 2)
                net_perf = it.get('NetworkInfo', {}).get('NetworkPerformance')

                rows.append({
                    'Region':               region,
                    'InstanceType':         it['InstanceType'],
                    'vCPUs':                it['VCpuInfo']['DefaultVCpus'],
                    'MemoryMiB':            mem_mib,
                    'RAM (GiB)':            ram_gib,
                    'NetworkPerformance':   net_perf,
                    'StorageInfo':          it.get('InstanceStorageInfo'),
                    'Accelerators':         it.get('GpuInfo') or it.get('FpgaInfo')
                })

    return pd.DataFrame(rows)


In [27]:
df_types = collect_all_instance_types()
# print(df_types.columns)
df_types.head()



,Region,InstanceType,vCPUs,MemoryMiB,RAM (GiB),NetworkPerformance,StorageInfo,Accelerators
0,ap-south-1,m6a.large,2,8192,8.0,Up to 12.5 Gigabit,None,None
1,ap-south-1,c6a.16xlarge,64,131072,128.0,25 Gigabit,None,None
2,ap-south-1,m5.8xlarge,32,131072,128.0,10 Gigabit,None,None
3,ap-south-1,c5n.2xlarge,8,21504,21.0,Up to 25 Gigabit,None,None
4,ap-south-1,m7gd.16xlarge,64,262144,256.0,30 Gigabit,"{'TotalSizeInGB': 3800, 'Disks': [{'SizeInGB':...",None
